In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential 
from keras.layers import Embedding,LSTM,Dense,GRU
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
stmts = pd.read_csv("../input/zindi-aldc/Train_df.csv")

stmts.isnull().values.any()


stmts.head()

In [ ]:
stmts.shape


In [ ]:
import seaborn as sns

sns.countplot(x='label', data=stmts)

In [ ]:
import re
def preprocess(sen):
   
    # Remove punctuations and numbers
    sen = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sen = re.sub(r"\s+[a-zA-Z]\s+", ' ', sen)

    # Removing multiple spaces
    sen = re.sub(r'\s+', ' ', sen)
    
    sen=sen.lower()

    return sen

In [ ]:
import numpy as np
encoding_dict={'Depression':0,'Drugs':1,'Suicide':2,'Alcohol':3}
y =stmts['label']
y = np.array(list(map(lambda x: encoding_dict[x], y)))
y[:5]

In [ ]:
X_df = stmts.drop(['label'],axis=1)
X_df.head()

In [ ]:
X=[]
for stmt in list(X_df['text']):
    X.append(preprocess(stmt))
X[:5]

In [ ]:
y[:5]

In [ ]:
from keras.utils import to_categorical

y_one_hot=to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.20, random_state=42)


In [ ]:
y_train[2]
# X_test[:10]
# X_train[0]

In [ ]:
from keras.preprocessing.text import Tokenizer


In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_token[0]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
maxlen = 10

X_train_pad = pad_sequences(X_train_token, padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_test_token, padding='post', maxlen=maxlen)


In [ ]:
X_train_pad[0]

In [ ]:
from keras.metrics import categorical_crossentropy
from keras.layers import Dropout, Dense, Flatten, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.models import Model


In [ ]:
hid1=5
hid2=25
hid3=125
embed=200
out= 4 
epoch=100
early=15
# sgd=SGD(lr=0.01)


def model():
    model=Sequential()
    model.add(Embedding(vocab_size, embed, input_length=maxlen))
    model.add(Flatten())
    model.add(Dense(hid2,activation ="sigmoid"))
#     model.add(Dropout(0.4))
    model.add(Dense(hid3,activation ="sigmoid"))
    model.add(Dense(out,activation ="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.005) , metrics=['accuracy'])
    return model

    



In [ ]:
model=model()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es_callback = EarlyStopping(monitor="val_loss", patience=20)
callbacks = [es_callback]

In [ ]:
history = model.fit(X_train_pad,
              y_train,
              epochs=100,
              batch_size=31,
              validation_data=(X_test_pad, y_test),
                                
             )

In [ ]:
model.save("adam005acc7661.h5")

In [ ]:
inferdata=pd.read_csv('../input/zindi-aldc/Test_df.csv')

In [ ]:
inferdata.head()

In [ ]:
infer=[]
for stmt in list(inferdata['text']):
    infer.append(preprocess(stmt))
infertoken = tokenizer.texts_to_sequences(infer)
inferpad = pad_sequences(infertoken, padding='post', maxlen=15)


In [ ]:
X_train_pad[:4]


In [ ]:
inferpad[:4]

In [ ]:
inferdata.shape

In [ ]:
test_score=model.evaluate(X_test_pad, y_test)
print(test_score)

In [ ]:
pred_index=model.predict(inferpad,batch_size=30)
predind = np.argmax(pred_index, axis=1)
predind[:10]